In [ ]:
import glob
import pandas as pd
import re
from Bio import Entrez
import time
import xml.etree.ElementTree as ET
import seaborn as sns; sns.set()
import copy
import numpy as np
from Bio import SeqIO

In [ ]:
# Edit the locations below to the location of your two results folders.

files_disease = glob.glob('ptyphi_results/*species.txt')
files_control = glob.glob('typhi_results/*species.txt')

location_of_fasta = 'databases/just_viruses/just_viruses.fasta'

In [ ]:
#List of sequences in the reference FASTA
fasta_ids = []

with open(location_of_fasta, "r") as handle:
    for record in SeqIO.parse(handle, "fasta") :
        fasta_ids.append(record.id)

In [ ]:
#Get read hit counts from disease folder

master_list = []

for file in files_disease:
    
    list_of_seqs = copy.deepcopy(fasta_ids)
    
    name = str(file)
    
    with open(file, "r") as ifile:
        
        for line in ifile:
            
            data = line.strip().split(' ')
            data = data+[name, 'ptyphi']
            
            list_of_seqs.remove(data[1])
            
            
            master_list.append(data)
            
    for seq in list_of_seqs:
        
        data = [0, seq, name, 'ptyphi']
        
        master_list.append(data)

In [ ]:
#Get read hit counts from control folder

for file in files_control:
    
    name = str(file)
    
    list_of_seqs = copy.deepcopy(fasta_ids)
    
    with open(file, "r") as ifile:
        
        for line in ifile:
            
            data = line.strip().split(' ')
            data = data+[name, 'typhi']
            master_list.append(data)
            
            
    for seq in list_of_seqs:
        
        data = [0, seq, name, 'typhi']
        
        master_list.append(data)

In [ ]:
df = pd.DataFrame(columns=['count', 'seq', 'file', 'type'], data= master_list)

In [ ]:
def get_srr_accession(df):
    """
    Extract the SRR number from the file_location field of the dataframe.
    """
    pattern = re.compile(r"[SED]RR[0-9]+")

    file = df['file']

    return re.search(pattern, file).group(0)

In [ ]:
df['acc'] = df.apply(get_srr_accession, axis=1)

In [ ]:
df['count'] = df['count'].astype(int)

In [ ]:
def get_spot_count(df, email, max_errors):
    """
    Use the Entrez API to get the spot (read) count for that SRR number.
    """

    srr_acc = df['acc']
    
    print (srr_acc)

    # get SRR id
    Entrez.email = email

    error_count =0

    while error_count < max_errors:

        try:

            handle = Entrez.esearch(db="sra",term=srr_acc)

            record = Entrez.read(handle)

            handle.close()

            srr_id = record["IdList"][0]

            break

        except:

            print ('error occured collecting ID ', srr_acc, error_count)

            time.sleep(10)

            error_count = error_count +1


    # get SRR summary

    error_count =0

    while error_count < max_errors:

        try:

            handle = Entrez.esummary(db='sra', id=srr_id)

            record = Entrez.read(handle)

            handle.close()

            my_xml = record[0]['Runs']

            # Parse XML
            xml_object = ET.fromstringlist(["<root>", my_xml, "</root>"])

            # Get total_spots (reads)
            for child in xml_object:

                if child.attrib['acc'] == srr_acc:

                    print (srr_acc, child.attrib['total_spots'])

                    return int(child.attrib['total_spots'])

        except:

            print ('error occured collecting spot count', srr_acc, error_count)

            time.sleep(10)

            error_count = error_count +1

In [ ]:
data2 = my_list = list(df['acc'].unique())

spot_df = pd.DataFrame(data2, columns=['acc'])

In [ ]:
spot_df['spot_count'] = spot_df.apply(get_spot_count, axis=1, args=['laitanawe@gmail.com', 3])

In [ ]:
spot_df.to_csv('spot_map.csv')

In [ ]:
read_count_dict ={}

for row in spot_df.iterrows():
    read_count_dict[(row[1]['acc'])] = row[1]['spot_count']

In [ ]:
def normalise(df):
    
    srr = df['acc']
    
    read_count = read_count_dict[srr]
    
    return read_count

In [ ]:
df['read_count'] = df.apply(normalise, axis=1)

In [ ]:
df.head()

In [ ]:
def normalise_values(df):
    
    return (df['count'] / df['read_count']) * 1000000

In [ ]:
df['normalised_count'] = df.apply(normalise_values, axis=1)

In [ ]:
df.to_csv('typhi.ptyphi.comparison.csv')

In [ ]:
sns.barplot(x='type', y='normalised_count', data=df)

In [ ]:
grouped = df.groupby(['type', 'seq']).mean()

In [ ]:
grouped['type2'] = grouped.index.levels[0][ grouped.index.labels[0]]

In [ ]:
pivoted = grouped.pivot_table(index='seq', values='normalised_count', columns='type2')

In [ ]:
def percent_diff(df):
    
    value1 = df['ptyphi']
    value2 = df['typhi']
    
    pctdiff = abs(value1-value2)/((value1+value2)/2)*100
    
    return pctdiff

In [ ]:
pivoted['pct_diff'] = pivoted.apply(percent_diff,axis=1)

In [ ]:
pivoted = pivoted.sort_values('pct_diff')


pivoted[['pct_diff']].plot(kind='bar', figsize =(75,25))

In [ ]:
def pct_change(df):
    
    value1 = df['ptyphi']
    value2 = df['typhi']
    
    
    return ((value2-value1) / value1)*100

In [ ]:
pivoted['pct_change'] = pivoted.apply(pct_change, axis=1)

In [ ]:
pivoted = pivoted.sort_values('pct_change')

In [ ]:
pivoted['pct_change'].plot(kind='bar', figsize =(200,25))

In [ ]:
pivoted

In [ ]:
#"Convert SAM files to BAM and bedGraph to calculate the coverage"

In [ ]:
import subprocess
with open('analysis/sam2bam2bed.txt') as f:
    command = " ".join(line.rstrip("\n") for line in f)
print (command)
subprocess.check_call(command.split()

In [ ]:
import subprocess
with open('analysis/coverage.txt') as f:
    command = " ".join(line.rstrip("\n") for line in f)
print (command)
subprocess.check_call(command.split()